# Zuschauerzahlen in der Super League seit 2003

In diesem Notebook möchte ich die Zuschauerzahlen der vergangenen 15 Jahren in der Super League erfassen und herausfinden, auf welchen Faktoren die Zuschauerzahlen in den verscheidenen Schweizer Stadien beruhen.


## Mögliche Fragestellungen:

- In welchem Stadion/welcher Stadt waren seit 2003 am meisten Zuschauer anwesend? 
- wie gross war der Sprung bei einem Stadionneubau?
- Welche Teams bringen in welchen Stadien besonders viele Zuschauer?
- Bei welchen Teams hängt die Zuschauerzahl stärker/weniger stark von der eigenen Leistung/Platzierung ab?
- Welchen Einfluss spielt das Wetter auf die Zuschauerzahlen?
- in welcher Runde gibt es die meisten Zuschauer?

## Grundsätzliches Vorgehen:

- Einlesen der Zuschauerzahlen aller Super-League-Spiele seit 2003 (seit Einführung des aktuellen Modus)
- Dataframe aller Spiele erstellen
- ausrechnen der "erwarteten Zuschauer" pro Spiel, in dem ich folgende Zuschauerzahl-Faktoren variiere und danach die beste Faktorenkombination auswähle. 1) aktuelle Stärke des Heimteams (letzte fünf Spiele), 2) aktuelle Platzierung des Heimteams, 3) Anzahl erzielter Tore des Heimteams in den vergangenen fünf Spielen, 4) Wichtigkeit des Spiels (in den sechs letzten Spielen: geht es um Meistertitel, Europa-League-Plätze, Abstieg?) 5) Samstags- oder Sonntagsspiel? 6) aktuelle Stärke des Auswärtsteams, 7) Wetter während des jeweiligen Spiels, 8) Entfernung der Spielorte
- Am meisten auf die Zuschauerzahlen dürfte schlicht und einfach die "Grundattraktivität" bzw. die Tradition eines Clubs ausmachen - ein sehr schwierig festzumachender Faktor. Damit die anderen zu erhebenden Faktoren (wie Wetter, Spielzeitpunt, Wichtigkeit) nicht durch diese "Grundattraktivität" eines gegnerischen Clubs verwässert werden, lege ich für jedes Team einen Zuschauergrundwert fest (z.B. Abweichung des Zuschauerschnitts, wenn Basel zu Gast ist vom total-Zuschauerschnitt )
- abspeichern der besten Daten in einem Team-Dataframe

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from datetime import datetime
from datetime import timedelta

## Alle Spiele einlesen
Auf "transfermarkt.com" mache ich Requests von allen Teams, die in den vergangenen 15 Jahren in der Super League spielten. Mit Beautifulsoup hole ich alle Daten heraus: Teams, Resultat, Zuschauer, Spielzeit, etc.

In [ ]:
total_spiele = []


vereinsliste = [["neuchatel-xamax-fcs/spielplan/verein/9084","Xamax"],["neuchatel-xamax/spielplan/verein/625","Xamax"], 
               ["fc-luzern/spielplan/verein/434","Luzern"],["fc-st-gallen-1879/spielplan/verein/257","St.Gallen"],
               ["fc-thun/spielplan/verein/938","Thun"],["grasshopper-club-zurich/spielplan/verein/504","Grasshoppers"],
               ["fc-vaduz/spielplan/verein/163","Vaduz"],["fc-basel-1893/spielplan/verein/26","Basel"],
               ["fc-zurich/spielplan/verein/260","Zürich"],["bsc-young-boys/spielplan/verein/452","Young Boys"],
               ["fc-lausanne-sport/spielplan/verein/527","Lausanne"],["servette-fc/spielplan/verein/61","Servette"],
               ["fc-sion/spielplan/verein/321","Sion"],["ac-bellinzona/spielplan/verein/2047","Bellinzona"], 
               ["fc-aarau/spielplan/verein/116","Aarau"],["fc-lugano/spielplan/verein/2790","Lugano"], 
               ["fc-schaffhausen/spielplan/verein/1277","Schaffhausen"],["yverdon-sport-fc/spielplan/verein/322","Yverdon"],
               ["fc-wil-1900/spielplan/verein/850","Wil"]]


for verein in vereinsliste:
    
    jahre = list(range(2003,2018))
    for jahr in jahre:
        soup = ""
        spielliste = []
        wettbewerb_art = []
        allespiele = ""
        
        url = "https://www.transfermarkt.ch/" + verein [0]+ "/plus/0?saison_id="+str(jahr)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        
        r = requests.get(url, headers=headers)
        print(url)
        

        soup = BeautifulSoup(r.text, 'html.parser')

        wettbewerb_art = soup.find_all('div', {'class':'box'})

        # Schlaufe, um alle Barragespiele auszuschliessen und nur die "regulären" Super-League-Spiele zu erhalten.
        for i in range (0,len(wettbewerb_art)):
            if "Super League" in wettbewerb_art[i].text and "Spielsystem" in wettbewerb_art[i].text and "Barrage" not in wettbewerb_art[i].text:
                allespiele = wettbewerb_art[i].text
                print(i)



        spielliste = allespiele.split("\t\t\t\t\t\t\t\t\n\n")
        
        for i in range(1,len(spielliste)): 
            spielliste[i] = spielliste[i].replace("So. ","Sun")
            spielliste[i] = spielliste[i].replace("Mo. ","Mon")
            spielliste[i] = spielliste[i].replace("Di. ","Tue")
            spielliste[i] = spielliste[i].replace("Mi. ","Wed")
            spielliste[i] = spielliste[i].replace("Do. ","Thu")
            spielliste[i] = spielliste[i].replace("Fr. ","Fri")
            spielliste[i] = spielliste[i].replace("Sa. ","Sat")
            spielliste[i] = spielliste[i].replace("\xa0\xa0"," ")

        
        #Schlaufe, um die einzelnen Werte (Teams, Resultat, etc.) herauszufiltern
        
        runde = 0
        for spiel in spielliste:
            if ".)" in spiel:
                spieldaten = spiel.split("\n")
                tag = spieldaten[0][0:3]
                datum = spieldaten[0].replace(" ","")[3:14]
                zeit = spieldaten[1]
                spielzeit = pd.to_datetime(tag + " " + datum + " " + zeit, format='%a %d.%m.%Y %H:%M')
                heim = spieldaten[2]
                if heim == "H":
                    heimteam = verein[1]
                    austeam = spieldaten[5][:-5]
                    heimrang = spieldaten[3][1:2]
                    ausrang = spieldaten[5][-3:-2]
                elif heim == "A":
                    heimteam = spieldaten[5][:-5]
                    austeam = verein[1]
                    heimrang = spieldaten[5][-3:-2]
                    ausrang = spieldaten[3][1:2]

                zuschauer = spieldaten[7].replace(".","")
                tore = spieldaten[8].replace(" ","").split(":")
                tore_heim = tore[0]
                tore_aus = tore[1]
                saison = str(jahr)+"/"+str(jahr+1)
                runde = runde + 1

                mini_dict = {"Saison":saison,
                             "Runde":runde,
                            "Tag":tag,
                            "Datum": datum,
                            "Zeit" : zeit,
                            "Spielzeit":spielzeit,
                            "Heimteam": heimteam,
                            "Rang Heimteam":heimrang,
                            "Auswärtsteam": austeam,
                            "Rang Auswärtsteam":ausrang,
                            "zuschauer":zuschauer,
                            "tore_heim":tore_heim,
                            "tore_aus":tore_aus}

                total_spiele.append(mini_dict)
       


## Spiel-Dataframe erstellen und speichern

In [ ]:
df = pd.DataFrame(total_spiele)

In [ ]:
df.to_csv('allespiele.csv')

In [ ]:
df = pd.read_csv("allespiele.csv")
#pd.options.display.max_rows = 6000

Da ich alle Spiele von jedem Team erfasst habe, sind die Spiele nun doppelt vorhanden. Dies werde ich ändern, muss dafür zunächst aber alle Teamnamen auf eine Schreibweise definieren:

In [ ]:
def changename(elem):
    
    elem = elem.lower()
    if 'basel' in elem:
        return 'Basel'
    if 'hoppers' in elem:
        return 'Grasshoppers'
    if 'wil' in elem:
        return 'Wil'
    if 'young' in elem:
        return 'YB'
    if 'st.gallen' in elem:
        return 'St.Gallen'
    if 'st. gallen' in elem:
        return 'St.Gallen'
    if 'zürich' in elem:
        return 'Zürich'
    if 'sion' in elem:
        return 'Sion'
    if 'servette' in elem:
        return 'Servette'
    if 'luzern' in elem:
        return 'Luzern'
    if 'thun' in elem:
        return 'Thun'
    if 'basel' in elem:
        return 'Basel'
    if 'aarau' in elem:
        return 'Aarau'
    if 'yverdon' in elem:
        return 'Yverdon'
    if 'xamax' in elem:
        return 'Xamax'
    if 'lausanne' in elem:
        return 'Lausanne'
    if 'bellinzona' in elem:
        return 'Bellinzona'
    if 'vaduz' in elem:
        return 'Vaduz'
    if 'lugano' in elem:
        return 'Lugano'
    if 'schaffhausen' in elem:
        return 'Schaffhausen'
    

In [ ]:
df['Heimteam'] = df['Heimteam'].apply(changename)
df['Auswärtsteam'] = df['Auswärtsteam'].apply(changename)

Nun lasse ich die Duplikate (die doppelt erfassten Spiele) verschwinden

In [ ]:
df = df.drop_duplicates(subset=['Heimteam', 'Auswärtsteam',"Saison","Spielzeit"], keep="first")

Und speichere das ganze geputzte Dataframe

In [ ]:
df.to_csv('allespiele_clean.csv')

# Wetterdaten einlesen 
Bevor ich die Spieldaten weiter bearbeite, möchte ich die Wetterdaten an das Spiel-Dataframe anhängen. Bzw für jedes Spiel soll nachher klar sein, wie warm es in der vollen Stunde des Spielbeginns war und wie viel es in den 10 Stunden vor dem Spiel geregnet hat, bzw ob es in der Stunde des Spielbeginns regnete (bzw schneite)

Die csv-Datei mit den Daten stammen von Meteo Schweiz (normalerweise kostenpflichtig, für Recherchen bzw. Ausbildungszwecken jedoch kostenlos)

In [6]:
#Wetterdaten einlesen
dfwetter = pd.read_csv("wetterdaten_seit_03.csv")


/Users/helenstreule/.virtualenvs/Ralfstreule/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
wetterortliste = set(list(dfwetter['Ort']))
wetterortliste

{'BAS', 'GVE', 'LUG', 'LUZ', 'SMA', 'STG', 'THU', 'stn'}

In [8]:
def change_ortname(ort):
    
    if ort == "BAS":
        return 'Basel'
    if ort == "GVE":
        return 'Genf'
    if ort == "LUG":
        return 'Lugano'
    if ort == "LUZ":
        return 'Luzern'
    if ort == "SMA":
        return 'Zürich'
    if ort == "STG":
        return 'St.Gallen'
    if ort == "THU":
        return 'Thun'

dfwetter['Ort'] = dfwetter['Ort'].apply(change_ortname)


In [9]:
dfwetter

,Ort,Zeit,Regen,Temperatur,Sonnenschein
0,Basel,2003010100,0.0,8.0,-
1,Basel,2003010101,0.0,7.8,-
2,Basel,2003010102,0.0,7.8,-
3,Basel,2003010103,0.0,7.9,-
4,Basel,2003010104,0.0,7.6,-
5,Basel,2003010105,0.0,7.4,-
6,Basel,2003010106,0.0,7.5,-
7,Basel,2003010107,0.0,7.4,-
8,Basel,2003010108,0.0,7.3,-
9,Basel,2003010109,0.0,8.0,-


In [10]:
# da die Wetterdaten für Thun in einer anderen Parameter-Reihenfolge geliefert wurden, ändere ich dies zunächst

thunwetter = dfwetter[dfwetter["Ort"] == "Thun"]
thunwetter["Speicher"] = thunwetter["Sonnenschein"]
thunwetter["Sonnenschein"] = thunwetter["Temperatur"]
thunwetter["Temperatur"] = thunwetter["Speicher"]
thunwetter


/Users/helenstreule/.virtualenvs/Ralfstreule/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/helenstreule/.virtualenvs/Ralfstreule/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/helenstreule/.virtualenvs/Ralfstreule/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,Ort,Zeit,Regen,Temperatur,Sonnenschein,Speicher
841404,Thun,2012121909,0.0,-,-,-
841405,Thun,2012121910,0.0,-,-,-
841406,Thun,2012121911,0.0,-,-,-
841407,Thun,2012121912,0.0,-,-,-
841408,Thun,2012121913,0.0,-,-,-
841409,Thun,2012121914,0.0,-,-,-
841410,Thun,2012121915,0.0,-,-,-
841411,Thun,2012121916,0.0,-,-,-
841412,Thun,2012121917,0.0,-,-,-
841413,Thun,2012121918,0.0,-,-,-


In [1]:
dfwetterneu = pd.merge(thunwetter, dfwetter, how='inner', on=['Ort'])
dfwetterneu

NameError: name 'pd' is not defined

In [ ]:
# um das Wetterdatenframe mit dem Spiel-Dataframe verbinden (mergen) zu können, 
# brauche ich vergleichbare Spalten - dies sind in diesem Fall das Spieldatum und die Zeit. 
# Hier treffe ich Vorbereitungen

# (Ich mache den Merge-Vergleich später über String-Spalten. Den Merge-Vergleich mit datatime-Format habe ich nicht 
# in den Griff gekriegt, da dort die Zeit bis auf die Minuten ausgeschrieben ist und es somit bei einem Vergleich 
# zwischen Wetter - und Spieldaten keine Übereinstimmungen gibt - wäre wohl einfacher lösbar?)

# hier erstelle ich eine Spalte mit dem String-Datum, wie sie auch im Spiel-Datenframe vorkommt:

def datumersteller(rohdatum):
    rohdatum = str(rohdatum)
    jahr = rohdatum[:4]
    monat = rohdatum[4:6]
    tag = rohdatum[6:8]
    
    if monat[:1] == "0":
        monat = monat[-1:]
        
    if tag[:1] == "0":
        tag = tag[-1:]
    datumneu = tag + "." + monat + "." + jahr
    return datumneu

dfwetter["Datum"] = dfwetter["Zeit"].apply(datumersteller)


In [ ]:
#und hier eine Spalte mit der vollen Stunde

def stunden_extrahierer(spielzeit):
    zeitstring = str(spielzeit)
    hour = zeitstring[-2:]
    if hour[:1] == "0":
        hour = hour[-1:]
        
    return hour

dfwetter["Stunde"] = dfwetter["Zeit"].apply(stunden_extrahierer)

In [ ]:
# nun mache ich weitere Wetterspalten, welche später wichtig sind für die Auswertung der Zuschauerzahlen
# Wie viel hat es in den fünf Stunden vor dem Spiel geregnet? Wie oft schien die Sonne in den vergangenen 
# zehn Stunden? 

In [ ]:
dfwetter

In [ ]:
def langzeitregen()
    df[team] = df[(df["Heimteam"] == team) | (df["Auswärtsteam"] == team)][team].rolling(min_periods=1, window=5).sum()

Spiel-Dataframe wieder einlesen:

In [ ]:
df = pd.read_csv("allespiele_clean.csv")

Auch für das Spiel-Datenframe bilde ich eine Spalte "Stunde":

In [ ]:
def stundencutter(zeit):
    neuezeit = zeit[:2]
    return neuezeit

df["Stunde"] = df["Zeit"].apply(stundencutter)

In [ ]:
# nun merge ich die beiden

In [ ]:
df

Um die aktuellen Stärken der Teams herauszufinden (Erfolge in den letzten fünf Spielen), muss ich zunächst berechnen, wieviele Punkte die Teams in jedem Spiel geholt haben. Das ist im df bisher nicht erfasst, sondern nur die Tore.

In [ ]:
def punkteheim_rechner(toreheim,toreaus):
    if toreheim > toreaus:
        punkte = 3
    if toreaus > toreheim:
        punkte = 0
    if toreheim == toreaus: 
        punkte = 1
    return punkte
        
def punkteaus_rechner(toreheim,toreaus):
    if toreheim < toreaus:
        punkte = 3
    if toreaus < toreheim:
        punkte = 0
    if toreheim == toreaus: 
        punkte = 1  
    return punkte

In [ ]:
df["punkteheim"] = df["punkteheim"].apply(punkteheim_rechner)
df["punkteaus"] = df["punkteaus"].apply(punkteaus_rechner)

In [ ]:
df["punkteaus"] = 0
df["punkteheim"] = 0
df['punkteaus'] = np.where((df['tore_aus'] > df['tore_heim']), df['punkteaus'] + 3, df["punkteaus"])
df['punkteheim'] = np.where((df['tore_heim'] > df['tore_aus']), df['punkteheim'] + 3, df["punkteheim"])
df['punkteaus'] = np.where((df['tore_aus'] == df['tore_heim']), df['punkteaus'] + 1, df["punkteaus"])
df['punkteheim'] = np.where((df['tore_aus'] == df['tore_heim']), df['punkteheim'] + 1, df["punkteheim"])

In [ ]:
# Liste der Vereine erstellen, um später durch die Vereinsliste iterieren zu können
teamliste = set(list(df['Heimteam']))

In [ ]:
#dito mit den Saisons
alle_saisons = set(list(df['Saison']))

Nun erstelle ich für jedes der Super-League-Teams der vergangenen 15 Jahre eine df-Spalte,... 

In [ ]:
for team in teamliste:
    df[team] = 0

...um darin nun die erreichten Punkte pro Spiel einsetzen zu können

In [ ]:
for team in teamliste:
    df[team] = np.where((df['Heimteam'] == team), df[team] + df['punkteheim'], df[team])
    df[team] = np.where((df['Auswärtsteam'] == team), df[team] + df['punkteaus'], df[team])
  

Auch dieses abgeänderte Dataframe speichere ich ab

In [ ]:
df = df.sort_values('Spielzeit')
df.to_csv('allespiele_punkte.csv')


In [ ]:
df = pd.read_csv("allespiele_punkte.csv")

Nun schreibe ich in die erstellte Teamspalte die erreichten Punkte der vergangenen fünf Spiele, was am ehesten eine Aussage über die aktuelle Verfassung eines Teams sagt.

In [ ]:
for team in teamliste:
    df[team] = df[(df["Heimteam"] == team) | (df["Auswärtsteam"] == team)][team].rolling(min_periods=1, window=5).sum()

Ich erstelle ein zweites Dataframe, in welchem ich alle Super-League-Teams aufliste und in den Spalten Werte wie den Heimspiel-Zuschauerschnitt gegen alle anderen Teams einsetze, später kommen verschiedene Faktoren dazu.

In [ ]:
df

In [ ]:
dfz = pd.DataFrame(columns=["Heimteam"])
for heimteam in teamliste:  
    dfz = dfz.set_value(len(dfz), 'Heimteam', heimteam)
for auswärtsteam in teamliste:
        dfz[auswärtsteam] = 0

In [ ]:
for heimteam in teamliste:
    for auswärtsteam in teamliste:
        #Zuschauerschnitt bei allen Spielen "heimteam gegen auswärtsteam"
        dfz.loc[dfz["Heimteam"] == heimteam, auswärtsteam] = df[(df["Heimteam"] == heimteam) & (df["Auswärtsteam"] == auswärtsteam)]["zuschauer"].mean()
        


In [ ]:
#noch unvollständig, Schnitt einer Reihe herausfinden!!!

for heimteam in teamliste:
    for auswärtsteam in teamliste:
        dfz.loc[dfz["Heimteam"] == heimteam, auswärtsteam] = dfz[dfz["Heimteam"] == heimteam]

In [ ]:
dfz

In [ ]:
dfz["Basel"].sum()

In [ ]:
#Beliebtheitsfaktor ausrechnen: Wieviele Zuschauer bringen die Teams ins Stadion?

for team in teamliste:
    for team in teamliste:
        dfz[team]

In [ ]:
eigenerrangfaktorliste = [20,50,100]
formfaktorliste = [20,50,100]
gegnerrangfaktorliste = [20,50,100]


#Durch die Teamliste iterieren, alle Heimspiele jedes Teams werden geprüft
for heimteam in teamliste:
    print(heimteam)
    
    # neues Dataframe für aktuell in der Schlaufe durchlaufenes Team erstellen, 
    df_heimteam = df[df["Heimteam"] == heimteam]
    df_heimteam["erwartetezuschauer"] = 0
    # Werte reseten
    mindifferenz = 1000000000000
    besterformfaktor = 0
    bestergegnerrangfaktor = 0
    bestereigenerrangfaktor = 0
    
    #alle für die Ausrechnung nötigen Werte setzen
    df_heimteam["aktuelle_stärke"] = (df_heimteam[heimteam] - 7.5) / 1.5
    df_heimteam["rangfaktor_aus"] = abs(df_heimteam["Rang Auswärtsteam"] - 10) - 5
    df_heimteam["rangfaktor_heim"] = abs(df_heimteam["Rang Heimteam"] - 10) -5
    #Grundwerte Zuschauerschnitt für jedes Auswärtsteam setzen (Grundattraktivität der Clubs im jeweilgen Stadion)
    for auswärtsteam in teamliste: 
        df_heimteam["teamzuschauerschnitt"] = dfz[dfz["Heimteam"] == heimteam][auswärtsteam]
    
      
    for eigenerrangfaktor in eigenerrangfaktorliste:
        for gegnerrangfaktor in gegnerrangfaktorliste:
            for formfaktor in formfaktorliste:
                df_heimteam["unterschied"] = 0
                
                #Herzstück: hier werden für alle Faktoren-Kombinationen die erwarteten Zuschauer ausgerechnet, wo 
                # es am besten zu den realen Zuschauerzahlen passt, werden die Faktoren gespeichert
                df_heimteam["erwartetezuschauer"] = (1 + df_heimteam["aktuelle_stärke"]/formfaktor) * (1 + (df_heimteam["rangfaktor_aus"])/gegnerrangfaktor) * (1 + (df_heimteam["rangfaktor_heim"])/eigenerrangfaktor) * df_heimteam["teamzuschauerschnitt"]
                df_heimteam["unterschied"] = abs(df_heimteam[df_heimteam["Heimteam"] == heimteam]["erwartetezuschauer"] - df_heimteam["zuschauer"])
                differenz = df_heimteam[df_heimteam["Heimteam"] == heimteam]["unterschied"].sum()
                if differenz < mindifferenz:
                    mindifferenz = differenz
                    besterformfaktor = formfaktor
                    bestergegnerrangfaktor = gegnerrangfaktor
                    bestereigenerrangfaktor = eigenerrangfaktor
    df.loc[df["Heimteam"] == heimteam, "erwartetezuschauer"] = (1 + df_heimteam["aktuelle_stärke"]/besterformfaktor) * (1 + (df_heimteam["rangfaktor_aus"])/bestereigenerrangfaktor) * (1 + (df_heimteam["rangfaktor_heim"])/bestergegnerrangfaktor) * df_heimteam["teamzuschauerschnitt"]
    df.loc[df["Heimteam"] == heimteam, "unterschied"] = abs(df_heimteam["erwartetezuschauer"] - df_heimteam["zuschauer"])
    
    print(str(bestereigenerrangfaktor)+ " " + str(besterformfaktor) + " " + str(bestergegnerrangfaktor))
    print(str(1 + df_heimteam["aktuelle_stärke"].max()/besterformfaktor))

In [ ]:
#dffcsgheim["erwartetezuschauer"] = (1 + dffcsgheim["aktuelle_stärke"]/besterformfaktor) * (1 + dffcsgheim["gegner-attraktivität"]/bestergegnerfaktor) * (1 + dffcsgheim["rangfaktor_aus"]/bestergegnerrangfaktor) * 11300
#dffcsgheim["unterschied"] = abs(dffcsgheim["erwartetezuschauer"] - dffcsgheim["zuschauer"])
#dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"unterschied"] = abs(dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["unterschied"])



df


In [ ]:
dffcsgheim

In [ ]:
for team in teamliste:
    dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"erwartetezuschauer"] = (1 + dffcsgheim["aktuelle_stärke"]/besterformfaktor) * (dffcsgheim["gegnerrang_grundfaktor"] ** bestergegnerrangfaktor + 1) * bestergegnerfaktor * gegnergrundfaktor * 11300
    dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"unterschied"] = dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["erwartetezuschauer"] - dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["zuschauer"]


In [ ]:
df_übersicht = dffcsgheim
df_übersicht

In [ ]:
#del df_übersicht["punkteaus"]
#del df_übersicht["punkteheim"]
#del df_übersicht["letztefünf"]
#df_übersicht

In [ ]:
dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == 'Basel', 'erwartetezuschauer'] = 20000

In [ ]:
dffcsgheim

In [ ]:
pd.options.display.max_rows = 3000

In [ ]:
spielliste = []
wettbewerb_art = []
allespiele = ""
url = "https://www.transfermarkt.ch/fc-st-gallen-1879/spielplan/verein/257/plus/0?saison_id=2011"
    #headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get(url, headers=headers)
print(url)
    
soup = BeautifulSoup(r.text, 'html.parser')

wettbewerb_art = soup.find_all('div', {'class':'box'})

    
for i in range (0,len(wettbewerb_art)):
    if "Super League" in wettbewerb_art[i].text and "Spielsystem" in wettbewerb_art[i].text:
            allespiele = wettbewerb_art[i].text
            print(i)
    
            
    
spielliste = allespiele.split("\t\t\t\t\t\t\t\t\n\n")
#print(spielliste)
for i in range(1,len(spielliste)): 
    spielliste[i] = spielliste[i].replace("So. ","Sun")
    spielliste[i] = spielliste[i].replace("Mo. ","Mon")
    spielliste[i] = spielliste[i].replace("Di. ","Tue")
    spielliste[i] = spielliste[i].replace("Mi. ","Wed")
    spielliste[i] = spielliste[i].replace("Do. ","Thu")
    spielliste[i] = spielliste[i].replace("Fr. ","Fri")
    spielliste[i] = spielliste[i].replace("Sa. ","Sat")
    spielliste[i] = spielliste[i].replace("\xa0\xa0"," ")
        
    
for spiel in spielliste:
    if ".)" in spiel:
        spieldaten = spiel.split("\n")
        print(spieldaten)
        tag = spieldaten[0][0:3]
        datum = spieldaten[0].replace(" ","")[3:14]
        zeit = spieldaten[1]
        spielzeit = pd.to_datetime(tag + " " + datum + " " + zeit, format='%a %d.%m.%Y %H:%M')
        heim = spieldaten[2]
        rang = spieldaten[3][1:2]
        gegner = spieldaten[5][:-5]
        rang_gegner = spieldaten[5][-3:-2]
        zuschauer = spieldaten[7]
        tore = spieldaten[8].replace(" ","").split(":")
        tore_heim = tore[0]
        tore_aus = tore[1]

        mini_dict = {"Tag":tag,
                    "Datum": datum,
                    "Zeit" : zeit,
                    "Spielzeit":spielzeit,
                    "heim": heim,
                    "rang":rang,
                    "gegner": gegner,
                    "rang_gegner":rang_gegner,
                    "zuschauer":zuschauer,
                    "tore":tore,
                    "tore_heim":tore_heim,
                    "tore_aus":tore_aus}
        
        total_spiele.append(mini_dict)


In [ ]:
for team in teamliste:
    for gegnerrangfaktor in gegnerrangfaktorliste:
        for formfaktor in formfaktorliste:
            for gegnerfaktor in gegnerfaktorliste:
                dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["unterschied"] = 0
                dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["erwartetezuschauer"] = 11300
                #dffcsgheim["erwartetezuschauer"] = (1 + dffcsgheim["aktuelle_stärke"]/formfaktor) * (1 + dffcsgheim["gegner-attraktivität"]/gegnerfaktor) * (1 + dffcsgheim["rangfaktor_aus"]/gegnerrangfaktor) * 11300
                #dffcsgheim["unterschied"] = abs(dffcsgheim["erwartetezuschauer"] - dffcsg["zuschauer"])
                dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"erwartetezuschauer"] = (1 + dffcsgheim["aktuelle_stärke"]/formfaktor) * (1 + dffcsgheim["gegner-attraktivität"]/gegnerfaktor) * (1 + dffcsgheim["rangfaktor_aus"]/gegnerrangfaktor) * 11300
                dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"unterschied"] = abs(dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["erwartetezuschauer"] - dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["zuschauer"])
                differenz = dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["unterschied"].sum()
                if differenz < mindifferenz:
                    mindifferenz = differenz
                    bestergegnerfaktor =  gegnerfaktor
                    besterformfaktor = formfaktor
                    bestergegnerrangfaktor = gegnerrangfaktor
    dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"erwartetezuschauer"] = (1 + dffcsgheim["aktuelle_stärke"]/besterformfaktor) * (1 + dffcsgheim["gegner-attraktivität"]/bestergegnerfaktor) * (1 + dffcsgheim["rangfaktor_aus"]/bestergegnerrangfaktor) * 11300
    dffcsgheim.loc[dffcsgheim["Auswärtsteam"] == team,"unterschied"] = abs(dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["erwartetezuschauer"] - dffcsgheim[dffcsgheim["Auswärtsteam"] == team]["zuschauer"])
    print(team)
    print(str(bestergegnerfaktor)+ " " + str(besterformfaktor) + " " + str(bestergegnerrangfaktor))


In [ ]:
wettbewerb_art[4]

In [ ]:
#allespiele

In [ ]:
team_maximalschnitt = df.groupby("Auswärtsteam")["zuschauer"].mean().sort_values().max()
team_minimalschnitt = df.groupby("Auswärtsteam")["zuschauer"].mean().sort_values().min()
team_minimalschnitt

df["gegner-attraktivität"] = 1
schritt = (team_maximalschnitt - team_minimalschnitt) / 10
for team in teamliste:   
    if df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = 5
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 2 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = 4
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 3 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = 3
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 4 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = 2
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 5 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = 1
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 6 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = - 1
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 7 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = - 2
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 8 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = - 3
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 9 * schritt):
        df.loc[df["Auswärtsteam"] == team,"gegner-attraktivität"] = - 4
    elif df[df["Auswärtsteam"] == team]["zuschauer"].mean() > (team_maximalschnitt - 10 * schritt):
        3